# [CDAF] Atividade 3

## Nome e matrícula
Nome: Luís Felipe Ramos Ferreira
Matrícula: 2019022553

## Referências
- [1] https://figshare.com/collections/Soccer_match_event_dataset/4415000
- [2] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.spadl.wyscout.convert_to_actions.html
- [3] https://github.com/TomDecroos/matplotsoccer
- [4] https://soccermatics.readthedocs.io/en/latest/gallery/lesson1/plot_PlottingShots.html
- [5] https://soccermatics.readthedocs.io/en/latest/gallery/lesson1/plot_PlottingPasses.html
- [6] https://soccermatics.readthedocs.io/en/latest/gallery/lesson1/plot_PassNetworks.html

## Questão 1
- Baixe o dataset 'Wyscout Europa Top 5 2017/2018' em [1].
- Escolha uma partida e carregue os dados de eventos em um dataframe do pandas.
- Converta os dados de eventos para SPADL [2].

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from socceraction import spadl
warnings.filterwarnings('ignore')

In [8]:
BARCELONA_VS_REAL_MADRID_ID: int = 2565907
BARCELONA_ID: int = 676 # home team id
spain_df: pd.DataFrame = pd.read_json("../data/atv03/events/events_Spain.json")
match_df: pd.DataFrame = spain_df[spain_df["matchId"] == BARCELONA_VS_REAL_MADRID_ID]

,type_id,subEventName,tags,player_id,positions,event_id,eventName,team_id,period_id,milliseconds,subtype_id,id
593455,8,Simple pass,[{'id': 1801}],3321,"[{'y': 49, 'x': 50}, {'y': 40, 'x': 37}]",2565907,Pass,675,1H,3.274992,85,249644096
593456,8,Simple pass,[{'id': 1801}],14723,"[{'y': 40, 'x': 37}, {'y': 23, 'x': 30}]",2565907,Pass,675,1H,5.109197,85,249644097
593457,8,Simple pass,[{'id': 1801}],3306,"[{'y': 23, 'x': 30}, {'y': 57, 'x': 26}]",2565907,Pass,675,1H,7.109958,85,249644098
593458,8,Simple pass,[{'id': 1801}],3309,"[{'y': 57, 'x': 26}, {'y': 40, 'x': 8}]",2565907,Pass,675,1H,8.912297,85,249644099
593459,8,Simple pass,[{'id': 1801}],3915,"[{'y': 40, 'x': 8}, {'y': 14, 'x': 11}]",2565907,Pass,675,1H,11.290220,85,249644100


In [10]:
match_df.rename(columns={"eventId": "type_id", "matchPeriod": "period_id", "subEventId": "subtype_id", "matchId": "game_id", "teamId": "team_id", "eventSec": "milliseconds", "playerId": "player_id"}, inplace=True)
match_df.head()

,type_id,subEventName,tags,player_id,positions,event_id,eventName,team_id,period_id,milliseconds,subtype_id,id
593455,8,Simple pass,[{'id': 1801}],3321,"[{'y': 49, 'x': 50}, {'y': 40, 'x': 37}]",2565907,Pass,675,1H,3.274992,85,249644096
593456,8,Simple pass,[{'id': 1801}],14723,"[{'y': 40, 'x': 37}, {'y': 23, 'x': 30}]",2565907,Pass,675,1H,5.109197,85,249644097
593457,8,Simple pass,[{'id': 1801}],3306,"[{'y': 23, 'x': 30}, {'y': 57, 'x': 26}]",2565907,Pass,675,1H,7.109958,85,249644098
593458,8,Simple pass,[{'id': 1801}],3309,"[{'y': 57, 'x': 26}, {'y': 40, 'x': 8}]",2565907,Pass,675,1H,8.912297,85,249644099
593459,8,Simple pass,[{'id': 1801}],3915,"[{'y': 40, 'x': 8}, {'y': 14, 'x': 11}]",2565907,Pass,675,1H,11.290220,85,249644100


In [ ]:
spadl_df: pd.DataFrame = spadl.wyscout.convert_to_actions(match_df, BARCELONA_ID)

In [ ]:
# cleaning up RAM
""" del spain_df """

## Questão 2
- Visualize uma sequência de 5 ações da partida usando matplotsoccer.actions [3].

## Questão 3
- Visualize os chutes da partida, desenvolvendo seu código em cima do dataframe SPADL. Faça um plot para cada time. Adapte de [4].
- Qual time as melhores chances da partida? Por quê?

## Questão 4
- Escolha um jogador da partida que você escolheu.
- Faça um heatmap de todas ações dele [3].
- Faça um heatmap de todas as ações ofensivas dele [3].
- Faça um heatmap de todas as ações defensivas dele [3].
- O que você pode inferir sobre o comportamento do jogador? O comportamento dele varia muito do ataque para a defesa?

In [ ]:
MESSI_ID: int = 3359
messi_df: pd.DataFrame = match_df[match_df["playerId"] == MESSI_ID]
messi_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id


## Questão 5
- Para o mesmo jogador, crie um mapa de passes com os passes que ele efetuou na partida, desenvolvendo seu código em cima do dataframe SPADL. Adapte de [5].
- O mapa de passes trouxe alguma informação nova sobre o jogador?

## Questão 6
- Crie uma rede de passes de cada uma das equipes, desenvolvendo seu código em cima do dataframe SPADL. Adapte de [6].
- O que você consegue inferir sobre a formação de cada equipe? Quais jogadores de cada equipe possuem o maior grau (tem maior soma do peso das arestas)?